In [1]:
import pandas as pd
import numpy as np

import os
import requests
import itertools
import time

from paths import DATA_FOLDER, PROCESSED_DATA_FOLDER, MODELS_FOLDER, ML_FOLDER
from BasicClasses import *

In [2]:
class ModelTesting(OllamaLLM):

    def __init__(self):
        super().__init__()

    def generation_CV(self, prompt, params_grid=None):
        if params_grid is None:
            params_grid = {
                "temperature": 1.0,
                "min_p": 0.01,
                "repeat_penalty": 1.0,
                "top_k": 64,
                "top_p": 0.95,
                "num_predict": 32768,
                "num_ctx": 16384
            }

        param_names = list(params_grid.keys())
        param_values = list(params_grid.values())
        all_combinations = list(itertools.product(*param_values))

        results = []

        for i, combination in enumerate(all_combinations, start=1):
            print(f'________________VARIANT #{i}________________')
            start = time.time()
            current = dict(zip(param_names, combination))
            print(f"Parameters: {current}")

            payload = {
                "model": self.model_name,
                "prompt": prompt,
                "stream": False,
                "options": {
                    "stop": [
                        "<end_of_turn>",
                        "<eos>"
                    ],
                    **current
                }
            }

            try:
                response = requests.post(self.api_url, json=payload, timeout=300)
                response.raise_for_status()
                end = time.time()
                print(f"Time: {end-start:.2f}s")
                time.sleep(0.5)
                results.append({
                    "id": i,
                    "parametres": current,
                    "response": response.json()["response"],
                    "success": True
                })
            except requests.exceptions.RequestException as e:
                print(f"Error calling Ollama API: {e}")
                results.append({
                    "id": i,
                    "parametres": current,
                    "response": str(e),
                    "success": False
                })
        return results



In [3]:
dq = DualQuery()
rag = LastFront()

Getting ChromaDB client...
Setting up an embedding function...
Getting classes...
Getting training data...
Setting up a reranker...
Connecting to Ollama...
Ollama server started
Done


In [4]:
retrieved_docs = dq("вивести ім'я страхувальника")
prompt = rag.prompt_generator(query="вивести ім'я страхувальника", retrieved_docs=retrieved_docs)

Compute Scores: 100%|██████████| 15/15 [00:00<00:00, 23.13it/s]


In [5]:
tester = ModelTesting()

In [6]:
params_grid = {
            "temperature": [0.7, 1, 1.3, 1.6],
            "min_p": [0.01, 0.05],
            "repeat_penalty": [1, 1.1],
            "top_k": [20, 32, 64, 96],
            "top_p": [0.4, 0.6, 0.8, 0.95],
            "num_predict": [32768],
            "num_ctx": [16384]
        }

In [7]:
results = tester.generation_CV(prompt, params_grid)

________________VARIANT #1________________
Parameters: {'temperature': 0.7, 'min_p': 0.01, 'repeat_penalty': 1, 'top_k': 20, 'top_p': 0.4, 'num_predict': 32768, 'num_ctx': 16384}
Time: 12.40s
________________VARIANT #2________________
Parameters: {'temperature': 0.7, 'min_p': 0.01, 'repeat_penalty': 1, 'top_k': 20, 'top_p': 0.6, 'num_predict': 32768, 'num_ctx': 16384}
Time: 3.10s
________________VARIANT #3________________
Parameters: {'temperature': 0.7, 'min_p': 0.01, 'repeat_penalty': 1, 'top_k': 20, 'top_p': 0.8, 'num_predict': 32768, 'num_ctx': 16384}
Time: 3.00s
________________VARIANT #4________________
Parameters: {'temperature': 0.7, 'min_p': 0.01, 'repeat_penalty': 1, 'top_k': 20, 'top_p': 0.95, 'num_predict': 32768, 'num_ctx': 16384}
Time: 3.03s
________________VARIANT #5________________
Parameters: {'temperature': 0.7, 'min_p': 0.01, 'repeat_penalty': 1, 'top_k': 32, 'top_p': 0.4, 'num_predict': 32768, 'num_ctx': 16384}
Time: 3.03s
________________VARIANT #6________________


In [8]:
results = pd.DataFrame(results)
results

,id,parametres,response,success
0,1,"{'temperature': 0.7, 'min_p': 0.01, 'repeat_pe...","__Result = ""Страхувальник є майновим поручител...",True
1,2,"{'temperature': 0.7, 'min_p': 0.01, 'repeat_pe...","__Result = ""Страхувальник є майновим поручител...",True
2,3,"{'temperature': 0.7, 'min_p': 0.01, 'repeat_pe...","__Result = ""Страхувальник є майновим поручител...",True
3,4,"{'temperature': 0.7, 'min_p': 0.01, 'repeat_pe...","__Result = ""Страхувальник є майновим поручител...",True
4,5,"{'temperature': 0.7, 'min_p': 0.01, 'repeat_pe...","__Result = ""Страхувальник є майновим поручител...",True
...,...,...,...,...
251,252,"{'temperature': 1.6, 'min_p': 0.05, 'repeat_pe...","__Result = ""Страхувальник є майновим поручител...",True
252,253,"{'temperature': 1.6, 'min_p': 0.05, 'repeat_pe...","__Result = ""Страхувальник є майновим поручител...",True
253,254,"{'temperature': 1.6, 'min_p': 0.05, 'repeat_pe...","__Result = ""Страхувальник є майновим поручител...",True
254,255,"{'temperature': 1.6, 'min_p': 0.05, 'repeat_pe...","__Result = ""Страхувальник є майновим поручител...",True
